<a href="https://colab.research.google.com/github/Hawkwellow/YCP_scrapping/blob/main/ycbscripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages and Libraries

In [1]:
import os
import sys
import json
import urllib
import shutil
import random
import glob
import time
import cv2
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
from urllib.request import Request, urlopen
import matplotlib.patches as patches

In [2]:
#shutil.rmtree("/content/mmdetection")

# Download YCB Image files


Define fuction to resize Images  Include both of .jpg and .pbm ( bitmap mask image ) 

In [3]:
def resize_img(imgpath):
  lf_img = Image.open(imgpath)
  resize = lf_img.resize((lf_img.width // 4, lf_img.height // 4))
  resize.save(imgpath)
  #plt.imshow(resize)
  return resize

In [4]:
#os.chdir

<function posix.chdir(path)>

In [5]:
# Define an output folder
output_directory = os.path.join("models", "ycb")
# Define a list of objects to download from
# http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/
#objects_to_download = "all"
objects_to_download = ["001_chips_can"]
#objects_to_download = ["001_chips_can", 
 #                       "002_master_chef_can",
 #                      "003_cracker_box",
 #                     "004_sugar_box", ]

# You can edit this list to only download certain kinds of files.
# 'berkeley_rgbd' contains all of the depth maps and images from the Carmines.
# 'berkeley_rgb_highres' contains all of the high-res images from the Canon cameras.
# 'berkeley_processed' contains all of the segmented point clouds and textured meshes.
# 'google_16k' contains google meshes with 16k vertices.
# 'google_64k' contains google meshes with 64k vertices.
# 'google_512k' contains google meshes with 512k vertices.
# See the website for more details.
#files_to_download = ["berkeley_rgbd", "berkeley_rgb_highres", "berkeley_processed", "google_16k", "google_64k", "google_512k"]
files_to_download = ["berkeley_rgb_highres"]

# Extract all files from the downloaded .tgz, and remove .tgz files.
# If false, will just download all .tgz files to output_directory
extract = True

base_url = "http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/data/"
objects_url = "https://ycb-benchmarks.s3.amazonaws.com/data/objects.json"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)


def fetch_objects(url):
    """ Fetches the object information before download """
    response = urlopen(url)
    html = response.read()
    objects = json.loads(html)
    return objects["objects"]


def download_file(url, filename):
    """ Downloads files from a given URL """
    u = urlopen(url)
    f = open(filename,"wb")
    file_size = int(u.getheader("Content-Length"))    
    print("Downloading: {} ({} MB)".format(filename, file_size/1000000.0))

    file_size_dl = 0
    block_sz = 65536
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break

        file_size_dl += len(buffer)
        f.write(buffer)
        status = r"%10d  [%3.2f%%]" % (file_size_dl/1000000.0, file_size_dl * 100. / file_size)
        status = status + chr(8)*(len(status)+1)
        print(status)
    f.close()
    

def tgz_url(object, type):
    """ Get the TGZ file URL for a particular object and dataset type """
    if type in ["berkeley_rgbd", "berkeley_rgb_highres"]:
        return base_url + "berkeley/{object}/{object}_{type}.tgz".format(object=object,type=type)
    elif type in ["berkeley_processed"]:
        return base_url + "berkeley/{object}/{object}_berkeley_meshes.tgz".format(object=object,type=type)
    else:
        return base_url + "google/{object}_{type}.tgz".format(object=object,type=type)


def extract_tgz(filename, dir):
    """ Extract a TGZ file """
    tar_command = "tar -xzf {filename} -C {dir}".format(filename=filename,dir=dir)
    os.system(tar_command)
    os.remove(filename)

def check_url(url):
    """ Check the validity of a URL """
    try:
        request = Request(url)
        request.get_method = lambda : 'HEAD'
        response = urlopen(request)
        return True
    except Exception as e:
        return False

def copytry(dep, dest):
  #handmade
  try:
    shutil.copy(dep, dest)
  except FileNotFoundError as e:
    pass



def download_datas():
  #handmade

    # Grab all the object information
    objects = fetch_objects(objects_url)

    

    # Download each object for all objects and types specified
    for object in tqdm(objects):
      if os.path.exists("/content/models/ycb/{object}_sampled".format(object = object)):
        continue
      if objects_to_download == "all" or object in objects_to_download:
          for file_type in files_to_download:
              url = tgz_url(object, file_type)
              if not check_url(url):
                  continue
              filename = "{path}/{object}_{file_type}.tgz".format(
                  path=output_directory,
                  object=object,
                  file_type=file_type)
              download_file(url, filename)
              time.sleep(1)
              if extract:
                  #画像ファイル(ex.N1_0.jpg)のパスとランダムに削除するためのリスト
                  #output_directory はycbまで
                  extract_tgz(filename, output_directory)
                  if object == "025_mug":
                    shutil.move("/content/models/ycb/home/bcalli/Desktop/025_mug", "/content/models/ycb")
                    shutil.rmtree("/content/models/ycb/home")

                    
                  imagepath = output_directory + "/{object}".format(object = object)

                  #Dataset中の外れ値（maskが欠けていたり）に対応、2022/11/02現在
                  if object == "063-c_marbles":
                    imagepath = output_directory + "/063-3_marbles"
                  if object == "071_nine_hole_peg_test":
                    imagepath = output_directory + "/071-1_nine_hole_peg_test"

                  imgfiles = glob.glob(imagepath + '/*.jpg')
                  if os.path.exists(imagepath + "/poses"):
                    shutil.rmtree(imagepath + "/poses") 
                  keep_list = random.sample(imgfiles, 100)

                  new_dir = output_directory + "/{object}_sampled".format(object = object)
                  #移動先のディレクトリを作成
                  os.makedirs(new_dir, exist_ok = True)
                  os.makedirs(new_dir + "/masks", exist_ok = True)
                  for tomove in keep_list:
                    #tomove:移動したいファイル　new_dir：移動先のディレクトリ
                    #移動したいファイルをリサイズしてから移動させる
                    
                    resize_img(tomove)
                    shutil.copy(tomove, new_dir)
                    #_sampledに移動したデータ(=keep_listに入ったデータ)に対応するmaskデータをmasksへとコピー
                    #keep_listの一つの要素のpath:
                    #'models/ycb/001_chips...N4_201.jpg'
                    #→'models/ycb/{objects}/masks/[tomoveから抽出文字列]_mask.pbmにする。
                    path_and_id = tomove.rsplit('/',1)
                    mask_file = "/" + path_and_id[1].replace('.jpg','') + "_mask.pbm"

                    

                    maskpath = imagepath + "/masks"
                    resize_img(maskpath + mask_file)
                    copytry(maskpath + mask_file, new_dir + "/masks")

              shutil.rmtree(imagepath)          
                    

In [6]:
if __name__ == "__main__":
  download_datas()

  0%|          | 0/103 [00:00<?, ?it/s]

ストリーミング出力は最後の 5000 行に切り捨てられました。
      1286  [79.70%]
      1286  [79.71%]
      1286  [79.71%]
      1286  [79.71%]
      1286  [79.72%]
      1286  [79.72%]
      1286  [79.73%]
      1286  [79.73%]
      1286  [79.73%]
      1286  [79.74%]
      1286  [79.74%]
      1286  [79.75%]
      1286  [79.75%]
      1287  [79.76%]
      1287  [79.76%]
      1287  [79.76%]
      1287  [79.77%]
      1287  [79.77%]
      1287  [79.78%]
      1287  [79.78%]
      1287  [79.78%]
      1287  [79.79%]
      1287  [79.79%]
  

  1%|          | 1/103 [03:01<5:08:14, 181.32s/it]

ストリーミング出力は最後の 5000 行に切り捨てられました。
      1273  [79.54%]
      1273  [79.54%]
      1273  [79.54%]
      1273  [79.55%]
      1273  [79.55%]
      1273  [79.56%]
      1273  [79.56%]
      1273  [79.57%]
      1273  [79.57%]
      1273  [79.57%]
      1273  [79.58%]
      1274  [79.58%]
      1274  [79.59%]
      1274  [79.59%]
      1274  [79.59%]
      1274  [79.60%]
      1274  [79.60%]
      1274  [79.61%]
      1274  [79.61%]
      1274  [79.61%]
      1274  [79.62%]
      1274  [79.62%]
      1274  [79.63%]
  

  2%|▏         | 2/103 [06:24<5:26:55, 194.22s/it]

ストリーミング出力は最後の 5000 行に切り捨てられました。
      1179  [78.26%]
      1179  [78.27%]
      1179  [78.27%]
      1179  [78.27%]
      1179  [78.28%]
      1179  [78.28%]
      1179  [78.29%]
      1179  [78.29%]
      1179  [78.30%]
      1179  [78.30%]
      1179  [78.31%]
      1179  [78.31%]
      1180  [78.31%]
      1180  [78.32%]
      1180  [78.32%]
      1180  [78.33%]
      1180  [78.33%]
      1180  [78.34%]
      1180  [78.34%]
      1180  [78.34%]
      1180  [78.35%]
      1180  [78.35%]
      1180  [78.36%]
  

  3%|▎         | 3/103 [09:29<5:16:19, 189.80s/it]

ストリーミング出力は最後の 5000 行に切り捨てられました。
      1274  [79.55%]
      1274  [79.56%]
      1274  [79.56%]
      1274  [79.56%]
      1274  [79.57%]
      1274  [79.57%]
      1274  [79.58%]
      1275  [79.58%]
      1275  [79.59%]
      1275  [79.59%]
      1275  [79.59%]
      1275  [79.60%]
      1275  [79.60%]
      1275  [79.61%]
      1275  [79.61%]
      1275  [79.61%]
      1275  [79.62%]
      1275  [79.62%]
      1275  [79.63%]
      1275  [79.63%]
      1275  [79.63%]
      1275  [79.64%]
      1275  [79.64%]
  

100%|██████████| 103/103 [12:43<00:00,  7.41s/it] 


In [7]:
#l = Image.open("/content/models/ycb/001_chips_can_sampled/masks/N1_12_mask.pbm")
#print(l)

preview sample mask file

In [8]:
def calc_bbox(pbm):
  lf_img = Image.open(pbm)
  lf_img_ary = np.array(lf_img, np.bool8)
  width, height = lf_img.size

  mask = ~lf_img_ary

  rows = np.any(mask, axis=1)
  cols = np.any(mask, axis=0)

  #bbox
  y_min, y_max = np.where(rows)[0][[0, -1]]
  x_min, x_max = np.where(cols)[0][[0, -1]]

  tmp=[]
  tmp.append(x_min)
  tmp.append(y_min)
  tmp.append(x_max - x_min)
  tmp.append(y_max - y_min)
  return tmp

#calc_bbox("/content/models/ycb/001_chips_can_sampled/masks/N1_117_mask.pbm")

In [9]:
def calc_area(pbm):
  lf_img = Image.open(pbm)
  lf_img_ary = np.array(lf_img, np.bool8)
  width, height = lf_img.size

  mask = ~lf_img_ary

  rows = np.any(mask, axis=1)
  cols = np.any(mask, axis=0)

  #bbox
  y_min, y_max = np.where(rows)[0][[0, -1]]
  x_min, x_max = np.where(cols)[0][[0, -1]]

  return (x_max - x_min) * (y_max - y_min)

In [10]:
#choose and change path to file that randomized
img = "/content/models/ycb/001_chips_can_sampled/masks/N1_108_mask.pbm"
lf_img = Image.open(img)
lf_img_ary = np.array(lf_img, np.bool8)
width, height = lf_img.size

mask = ~lf_img_ary

rows = np.any(mask, axis=1)
cols = np.any(mask, axis=0)

#bbox
y_min, y_max = np.where(rows)[0][[0, -1]]
x_min, x_max = np.where(cols)[0][[0, -1]]

rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                 linewidth=2,
                                 edgecolor=(0, 1, 0, 1),
                                 facecolor='none')
bbox = []
b = {}

plt.imshow(lf_img)
ax = plt.gca()
ax.add_patch(rect)

plt.show()

FileNotFoundError: ignored

Method to calcurate bbox rocation data from single .pbm data

In standard, bbox is described in [ xmin, ymin, bbox_width, bbox_height ]

In [ ]:
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [ ]:
a = Image.open("/content/models/ycb/001_chips_can_sampled/N2_171.jpg")

plt.imshow(a)

# Convert Datas to Dataset in COCO Format

To make json COCO format, prepare method to fill each dict value

In [11]:
def makeImagedict(samplename, id):
  tmp = {}
  imgpath = "/content/models/ycb/{samplename}/{id}.jpg".format(samplename = samplename, id = id)
  lf_img = Image.open(imgpath)
  width, height = lf_img.size
  filename = imgpath
   
  tmp['file_name'] = filename
  tmp['height'] = height
  tmp['width'] = width
  tmp['image_id'] = id
  return tmp

def makeAnnotationdict(samplename, id, category_num):
  tmp = {}
  
  tmp['image_id'] = id
  tmp['category_id'] = category_num
  tmp['bbox'] = calc_bbox("/content/models/ycb/{samplename}/masks/{id}_mask.pbm".format(samplename = samplename, id = id))
  tmp['area'] = calc_area("/content/models/ycb/{samplename}/masks/{id}_mask.pbm".format(samplename = samplename, id = id))
  tmp['segmentation'] = []
  tmp['iscrowd'] = 0
  return tmp

def makeCategorydict(samplename, category_num):
  tmp = {}
  categoryname = samplename[0:-8]

  tmp['category_id'] = category_num
  tmp['name'] = categoryname
  return tmp

In [12]:
if __name__ == "__main__":
  #samplepath = "/content/models/ycb/004_sugar_box_sampled"
  #get name of ~sampled

  samplelist = glob.glob("/content/models/ycb" + "/**")
  #samplelist →directory + category + _sampled というテキスト
  datajson = {}

  images = []
  annotations = []
  categories = []

  category_num = 1
  for sampleid in tqdm(samplelist):
    samplename = sampleid.rsplit('/',1)[1]
    categoryname = samplename[0:-8]
    imagelist = glob.glob(sampleid + "/*.jpg")
    masklist = glob.glob(sampleid + "/masks/*.pbm")
    idlist = []

    for image in imagelist:
      idlist.append(image.rsplit('/',1)[1][0:-4])

    for id in idlist:
      try:
        #add_dict = {}
        #add_dict["bbox"]  = calc_bbox("/content/models/ycb/{samplename}/masks/{id}_mask.pbm".format(samplename = samplename, id = id))
        #add_dict["image"] = "/content/models/ycb/{samplename}/{id}.jpg".format(samplename = samplename, id = id)
        #add_dict["label"] = categoryname
        images.append(makeImagedict(samplename, id))#list of each image dict
        annotations.append(makeAnnotationdict(samplename, id, category_num))#list of each annotation dict
        categories.append(makeCategorydict(samplename, category_num))#list of each category dict

      except FileNotFoundError:
        pass
    
    category_num+=1

  datajson['images'] = images
  datajson['annotations'] = annotations
  datajson['categories'] = categories
print(datajson)

cocoannotations = datajson


100%|██████████| 4/4 [00:01<00:00,  3.08it/s]

{'images': [{'file_name': '/content/models/ycb/001_chips_can_sampled/N5_246.jpg', 'height': 712, 'width': 1068, 'image_id': 'N5_246'}, {'file_name': '/content/models/ycb/001_chips_can_sampled/N5_45.jpg', 'height': 712, 'width': 1068, 'image_id': 'N5_45'}, {'file_name': '/content/models/ycb/001_chips_can_sampled/N4_213.jpg', 'height': 712, 'width': 1068, 'image_id': 'N4_213'}, {'file_name': '/content/models/ycb/001_chips_can_sampled/N4_78.jpg', 'height': 712, 'width': 1068, 'image_id': 'N4_78'}, {'file_name': '/content/models/ycb/001_chips_can_sampled/N5_282.jpg', 'height': 712, 'width': 1068, 'image_id': 'N5_282'}, {'file_name': '/content/models/ycb/001_chips_can_sampled/N3_105.jpg', 'height': 712, 'width': 1068, 'image_id': 'N3_105'}, {'file_name': '/content/models/ycb/001_chips_can_sampled/N4_45.jpg', 'height': 712, 'width': 1068, 'image_id': 'N4_45'}, {'file_name': '/content/models/ycb/001_chips_can_sampled/N1_345.jpg', 'height': 712, 'width': 1068, 'image_id': 'N1_345'}, {'file_nam

## json export

In [ ]:
import json

dir = "/content/_cocoannotations.json"
with open(dir, mode = 'wt', encoding = 'utf-8') as f:
  json.dump(datajson, f,  ensure_ascii=False, indent=2)

# Install MMDetection

In [13]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.4 MB/s eta 0:14:25tcmalloc: large alloc 1147494400 bytes == 0x39048000 @  0x7f760cc77615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████▌               | 1055.7 MB 1.4 MB/s eta 0:11:38tcmalloc: large alloc 1434370048 bytes == 0x7d69e000 @  0x7f760cc77615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
      -O checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

In [ ]:
map_location=torch.device('cpu')
torch.load

In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector

# Choose to use a config and initialize the detector
config = 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# Set the device to be used for evaluation
device='cuda:0'

# Load the config
config = mmcv.Config.fromfile(config)
# Set pretrained to be None since we do not need pretrained model here
config.model.pretrained = None

# Initialize the detector
model = build_detector(config.model)

# Load checkpoint
checkpoint = load_checkpoint(model, checkpoint, map_location=device)

# Set the classes of models for inference
model.CLASSES = checkpoint['meta']['CLASSES']

# We need to set the model's cfg for inference
model.cfg = config

# Convert the model to GPU
model.to(device)
# Convert the model into evaluation mode
model.eval()

#Make Dataset Class



In [ ]:

# the new config inherits the base configs to highlight the necessary modification
_base_ = './faster_rcnn_r50_fpn_1x_coco.py'

# 1. dataset settings
dataset_type = 'CocoDataset'
classes = fetch_objects(objects_url)
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file='path/to/your/train/annotation_data',
        img_prefix='path/to/your/train/image_data'),
    val=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file='path/to/your/val/annotation_data',
        img_prefix='path/to/your/val/image_data'),
    test=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file='path/to/your/test/annotation_data',
        img_prefix='path/to/your/test/image_data'))

# 2. model settings

# explicitly over-write all the `num_classes` field from default 80 to 5.
model = dict(
    roi_head=dict(
        bbox_head=[
            dict(
                type='Shared2FCBBoxHead',
                # explicitly over-write all the `num_classes` field from default 80 to 5.
                num_classes=103),
            dict(
                type='Shared2FCBBoxHead',
                # explicitly over-write all the `num_classes` field from default 80 to 5.
                num_classes=103),
            dict(
                type='Shared2FCBBoxHead',
                # explicitly over-write all the `num_classes` field from default 80 to 5.
                num_classes=103)],
    # explicitly over-write all the `num_classes` field from default 80 to 5.
    mask_head=dict(num_classes=5)))

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')